## Eksperiment 3 eeltöötlus

Erinevus eksperiment kahest:
* Juurde on lisatud ükskilausete korpus (tehtud sarnane eeltöötlus kui otsekõne ja jutustaja korpustele)
* Varasemalt mitte kasutatud testhulgast on lauseid validatsioonihulka juurde lisatud, et see oleks stabiilsem.

Kokku lauseid treeninghulgas:
Kokku lauseid validatsioonihulgas:


1. Üksiklausete tekstide ja wav-failide eraldamine kaustadesse

In [ ]:
import os
import shutil
import re
from sklearn.model_selection import train_test_split

In [ ]:
sentences_1 = ""
sentences_2 = ""
yksiklaused_wavs = ""

yksiklaused_txt_path = ""

In [ ]:
for line in lines:
    if line.str.startswith('eki_et'):
        line = re.sub('\t([0-9:]+)?\t\t', '\tx\t', line)
        parts = line.strip().split('\tx\t')
        assert len(parts) == 2
        wav_file_name = parts[0]
        sentence = parts[1]

        txt_file_name = wav_file_name.split('.wav')[0] + '.txt'
        with open(yksiklaused_txt_path + f'/{txt_file_name}', 'w', encoding='utf-8') as fout:
            fout.write(sentence)

2. Üksiklausete failide ümbernimetamine

In [ ]:
yksiklaused_txt_path = ""
yksiklaused_wav_path = ""

In [ ]:
for filename in os.listdir(yksiklaused_wav_path):
    if filename.endswith('wav'):
        # 'eki_et_ptr_10015.wav'
        orig_fname = filename.split('.')[0]
        no = orig_fname.split('_')[-1]
        new_name = 'yksiklause_' + no
        shutil.copyfile(os.path.join(yksiklaused_wav_path, filename), os.path.join(yksiklaused_wav_path, f'{new_name}.wav'))

        # Üksiklausete tekstifailide ümbernimetamine
        txt_filename = orig_fname + '.txt'
        shutil.copyfile(os.path.join(yksiklaused_txt_path, txt_filename), os.path.join(yksiklaused_txt_path, f'{new_name}.txt'))

3. Liiga väikeste ja suurte failide eemaldamine

In [ ]:
sample_file_selection = [] # Files from the folder that are in certain size range
files_discarded = 0
files_included = 0

for file in os.listdir(yksiklaused_wav_path):
    fsize = os.path.getsize(os.path.join(yksiklaused_wav_path, file))
    if fsize < 44000 or fsize > 1000000:
        files_discarded += 1 
        continue
    files_included += 1
    sample_file_selection.append(file)

print(f'{files_included} files included in datasets. {files_discarded} files discarded.')

4. Treeningandmete jagamine ja teiste treeningandmetega kokku liitmine

In [ ]:
train, val = train_test_split(sample_file_selection, train_size=0.9, shuffle=True, random_state=5)

5. Üksiklauste lisamine treening- ja validatsioonihulka

In [ ]:
def read_sentence_from_file(sentence: str, txt_folder):
    if not sentence.endswith('.txt'):
        sentence = sentence + '.txt'
    with open(os.path.join(txt_folder, sentence)) as f:
        sentence_text = f.read() + '\n'
    return sentence_text

def compose_txt_for_training(wav_file_subset, wav_file_reference_path, txt_folder):
    lines = []
    for fname in wav_file_subset:
        sentence_id = fname.split('.')[0]
        wav_reference = f'{wav_file_reference_path}/{fname}'
        sentence = read_sentence_from_file(sentence_id, txt_folder)
        lines.append(f'{wav_reference}|{sentence}')
    
    return lines

In [ ]:
# TODO: lisada pathid
import random

train_filelist_fname = 'est_audio_text_train_filelist_4400_1000000.txt'
new_train_path = ''

with open(train_filelist_fname) as f:
    lines = f.readlines()
    train = compose_txt_for_training(train, yksiklaused_wav_path, yksiklaused_txt_path)
    train_set = lines + train
    random.shuffle(train_set)
    with open(new_train_path, 'w', encoding='utf-8') as fout:
        fout.writelines([l + '\n' for l in train_set])

In [ ]:
validation_filelist_fname = 'est_audio_text_val_filelist_4400_10000000.txt'
test_filelist_fname = 'est_audio_text_test_filelist_4400_10000000.txt'

# 1. Lisa üksiklausete validatiooniread

# 2. Lisa  testandmestikust lisaread

# 3. Andmed juhuslikku järjekorda
